In [1]:
################################################################################
# Author 1:      Bernhard Lugger
# MatNr 1:       01114792
# Author 2:      Paula Nauta
# MatNr 2:       11938311
# File:          assignment2.ipynb
# Description:   several different plots of OECD data
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [2]:
# ========== imports ============

import pandas as pd
import pycountry as pc
import plotly.express as px

# import matplotlib.pyplot as plt # or from matplotlib import pyplot as plt
# import matplotlib.dates as mdates # we use this to space our date-xticks
# import numpy as np

# Plotly express is especcialy useful if you want to plot dataframes (e.g. Pandas Dataframe).
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

In [99]:
# ========== functions ============

def countries_fullname(country):
    """receives alpha_3 from a country (e.g. 'AUS') and returns the
    full name of the country (e.g. 'Australia')"""
    country_full = pc.countries.get(alpha_3=country)
    country_name = country_full.name
    return country_name


    # '*' means that you can pass as many parameters (in this case 'countries' as you wish)
def load_oecd_csv_to_df(file_path, *chosen_countries):
    """loads one csv from the given path and returns the relevant columns of arbitrary chosen
    country-alpha3 names to a dataframe. With LOCATION of countries in fullname. """
    oecd_data = pd.read_csv(file_path, header=0, usecols=['LOCATION','SUBJECT', 'MEASURE', 'TIME', 'Value'],
                            delimiter=",")     # ('header=0' --> column names)

    final_filtered_df = pd.DataFrame()
    country_alpha3_names = []

    for country in chosen_countries:
        country_alpha3_names.append(country)

    for country in country_alpha3_names:
        df_mask=oecd_data['LOCATION']==country            # known from assignment1
        filtered_df = oecd_data[df_mask]
        final_filtered_df = final_filtered_df.append(filtered_df)

    for country in chosen_countries:        # replace alpha3 names with full names of countries
        full_name = countries_fullname(country)
        final_filtered_df = final_filtered_df.replace([country], full_name)

    return final_filtered_df


def load_oecd_csv_to_df_alpha3(file_path, *chosen_countries):
    """loads one csv from the given path and returns the relevant columns of arbitrary
    chosen country-alpha3 codes to a dataframe. With LOCATION of countries in alpha3 name."""
    oecd_data = pd.read_csv(file_path, header=0, usecols=['LOCATION','SUBJECT', 'MEASURE', 'TIME', 'Value'],
                            delimiter=",")

    final_filtered_df = pd.DataFrame()
    country_alpha3_names = []

    for country in chosen_countries:
        country_alpha3_names.append(country)

    for country in country_alpha3_names:
        df_mask=oecd_data['LOCATION']==country
        filtered_df = oecd_data[df_mask]
        final_filtered_df = final_filtered_df.append(filtered_df)

    return final_filtered_df


def filter_dataframe(df, column, value):
    """returns only data where the given value is in column."""
    df_mask=df[column]==value
    filtered_df = df[df_mask]
    return filtered_df


def prepare_scatterplot1_df(df1, df2, year):
    """prepares/edits data from 2 dataframes from one year [int] to one new dataframe"""
    df1_filtered = filter_dataframe(df1, 'SUBJECT', 'GHG')
    df1_filtered = filter_dataframe(df1_filtered, 'MEASURE', 'TONNE_CAP')
    df1_filtered = filter_dataframe(df1_filtered, 'TIME', year)

    df1_filtered_indexed = df1_filtered.set_index('LOCATION')
    df1_filtered_indexed = df1_filtered_indexed.rename(columns={'Value': 'Values1'})
    df1_final = df1_filtered_indexed.Values1


    df2_filtered = filter_dataframe(df2, 'SUBJECT', 'TRY')
    df2_filtered = filter_dataframe(df2_filtered, 'TIME', year)

    df2_filtered_indexed = df2_filtered.set_index('LOCATION')
    df2_filtered_indexed = df2_filtered_indexed.rename(columns={'Value': 'Values2'})
    df2_final = df2_filtered_indexed.Values2

    df_final = pd.concat([df1_final, df2_final], axis=1) # <-- works but i guess not clean
    return df_final


def plot_lineplot(df, title, xaxis_text):
    """plots a lineplot of the given DataFrame with the given title and x-axis text."""
    fig = px.line(df, x="TIME",y="Value", color="LOCATION", title=title)
    fig.update_layout(xaxis={"rangeslider": {"visible": True}})
    fig.update_yaxes(title_text = xaxis_text)
    fig.show()


def plot_barplot(df, title, xaxis_text):
    """plots a barplot of the given DataFrame with the given title and x-axis text."""
    fig = px.bar(df, x="LOCATION",y="Value", color="LOCATION", title=title)
    fig.update_yaxes(title_text = xaxis_text)
    fig.show()


def plot_scatterplot(df):
    fig = px.scatter(df, x="Values1", y="Values2")
    # size='petal_length', hover_data=['petal_width'],color="LOCATION
    fig.show()

In [106]:
# =============== Plot functions ==================

# ----- Line-Plots --------
# Government researchers
def plot_lineplot1():
    """plots lineplot1 about Government researchers. """
    df_researchers = load_oecd_csv_to_df('datasets/DP_LIVE_02012022214642229.csv', 'AUT', 'ITA', 'AUS', 'USA', 'BEL')
    df_researchers_filtered = filter_dataframe(df_researchers, 'SUBJECT', 'TOT')
    df_researchers_filtered = filter_dataframe(df_researchers_filtered, 'MEASURE', 'PC_NATIONAL')
    plot_lineplot(df_researchers_filtered,"Government researchers","% of national total")

# Young population
def plot_lineplot2():
    """plots lineplot2 about Young population. """
    df_young = load_oecd_csv_to_df('datasets/DP_LIVE_02012022233800555.csv', 'AUT', 'ITA', 'AUS', 'USA', 'BEL')
    plot_lineplot(df_young,"Young population", "% of population")

# Fertility
def plot_lineplot3():
    """"plots lineplot3 of fertility rate"""
    df_fertility = load_oecd_csv_to_df('datasets/renamed/Fertility_rates.csv','AUT', 'ITA', 'AUS', 'USA', 'BEL')
    plot_lineplot(df_fertility,"Fertility Rate", "average number of children per women")



# ----- Bar-Plots --------
# Discrimination in the family
def plot_barplot1():
    """plots barplot1 about Attitudes towards working mothers. """
    df_discrimination = load_oecd_csv_to_df('datasets/DP_LIVE_03012022021117254.csv', 'AUT', 'DEU', 'ITA', 'AUS', 'USA', 'BEL')
    df_discrimination_filtered = filter_dataframe(df_discrimination, 'SUBJECT', 'ATTWORKMUM')
    plot_barplot(df_discrimination_filtered, "Discrimination in the family 2019", "Attitudes towards working mothers [%]")

# Income Inequality
def plot_barplot2():
    """plots barplot2 about income inequality in 2018 of different countries"""
    df_income_inequality = load_oecd_csv_to_df('datasets/renamed/Income_inequality.csv', 'AUT', 'DEU', 'ITA', 'AUS', 'BEL')
    df_income_inequality_filtered = filter_dataframe(df_income_inequality, 'TIME', 2018)
    #print (df_income_inequality_filtered)
    df_income_inequality_filtered = filter_dataframe(df_income_inequality_filtered, 'SUBJECT','GINI')
    plot_barplot(df_income_inequality_filtered, "Income Inequality", "Gini Coefficient")

# Poverty Gap
def plot_barplot3():
    """plots barplot3 about the poverty rate in 2019 of different countries"""
    df_poverty_gap = load_oecd_csv_to_df('datasets/renamed/Poverty_gap.csv', 'AUT', 'DEU', 'CZE', 'NLD', 'NOR', 'ESP')
    df_poverty_gap_filtered = filter_dataframe(df_poverty_gap, 'TIME', 2019)
    df_poverty_gap_filtered = filter_dataframe(df_poverty_gap_filtered, 'SUBJECT','18_65')
    plot_barplot(df_poverty_gap_filtered, "Poverty Gap", "Ratio of income below the poverty line")

# ----- Scatter-Plots --------
# Greenhouse gas (GHG), Tonnes/capita;   Adult education level - Tertiary, % of 25-64 year-olds, 2020 or latest available
def plot_scatterplot1():
    """plots scatterplot1 about Greenhouse gas (GHG), Tonnes/capita and  Adult education level - Tertiary. """
    df_ghg = load_oecd_csv_to_df_alpha3('datasets/DP_LIVE_04012022042348960.csv', 'AUT', 'ITA', 'AUS', 'USA', 'BEL')
    df_edu_ter = load_oecd_csv_to_df_alpha3('datasets/DP_LIVE_04012022044322787.csv', 'AUT', 'ITA', 'AUS', 'USA', 'BEL')
    df_scatter1 = prepare_scatterplot1_df(df_ghg, df_edu_ter, 2010)
    plot_scatterplot(df_scatter1)

In [107]:
# =========== final program ============
plot_lineplot1()
plot_lineplot2()
plot_lineplot3()
plot_barplot1()
plot_barplot2()
plot_barplot3()
plot_scatterplot1()